# Création d'un fichier BIOES pour évaluation avec la librairie NERVAL

Dans le but de tester la [librairie python NERVAL](https://teklia.com/blog/202104-nerval/) (Teklia), on utilise le modèle de reconnaissance générique français de Stanza pour générer un fichier BIOES.

Un fichier BIOES est un format d'annotation pour les tokens, notamment pour les tâches de reconnaissance d'entités nommées.

Il prend la forme suivante : 

`token tag`

Les tags sont les suivants : 

* B : beginning
* I : inside
* O : outside
* E : Ending
* S : Single element

Pour la phrase suivante "Léon III l'Isaurien était un empereur byzantin, né à Germanicia, en Turquie actuelle.", on aurait :

```
Léon B-PER
III I-PER
l' I-PER
Isaurien E-PER
était O
un O
empereur O
byzantin O
, O
né O
à O
Germanicia S-LOC
, O
en O
Turquie S-LOC
actuelle O
. O
```

## Import

On importe Stanza.

In [1]:
import stanza

## Création de la chaîne de traitement TAL

Pour effectuer la tâche de reconnaissance d'entités nommées, on crée la chaîne de traitement TAL en indiquant la langue, dont le modèle a préaleblement été téléchargé, ainsi que les tâches dont on a besoin : tokenization, et reconnaissance d'entités nommées.

In [2]:
nlp = stanza.Pipeline(lang='fr', processors='tokenize, ner')

2021-08-03 16:31:55 WARNING: Language fr package default expects mwt, which has been added
2021-08-03 16:31:55 INFO: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| ner       | wikiner |

2021-08-03 16:31:55 INFO: Use device: cpu
2021-08-03 16:31:55 INFO: Loading: tokenize
2021-08-03 16:31:55 INFO: Loading: mwt
2021-08-03 16:31:55 INFO: Loading: ner
2021-08-03 16:31:57 INFO: Done loading processors!


## Pré-traitement des données

A partir de notre échantillon, une page d'un répertoire de notaire résultant de l'HTR, avec un CER de 13%, on crée une liste où chaque index est un enregistrement. De la sorte, on crée une segmentation artificielle des phrases.

In [4]:
with open("input/FRAN_0025_5038_L-0_htr_cer_13.txt", "r", encoding="UTF-8") as fh:
    sample = fh.read()

In [5]:
entries_list = sample.split('¬')
entries_list_without_linebreakes = [entry.strip('\n') for entry in entries_list]
entries_list_without_linebreakes = entries_list_without_linebreakes[:-1]

In [16]:
entries_list_without_linebreakes

['An 1911, mois de Mai',
 'Gauvin- des reprises de M^me Suzanne Louise farnet spe au Mans (Sarthe) rue dus chalets 31 épouse séparée de corps de M. Auguste Aleandre',
 "Gauvin de l'Etat Liquidatif sur enonce",
 "Tricon (au prêt consenti par le Crédit foncier à Benjamin Joseph Maurice 1 rue de Paris boulevard de Courcelles 50 d'un prêt de 90000",
 'Roge par M^me Charlotte Amelia Félinte Buron proprsétaire a Paris rue Leon Coqniet 11 veuf Joeph Wilfrid Arnand en bl. pour interveur a uacte secession et detransport',
 'Dalenne (par M^me Syanne Nathalie Rogé rentière a Paris rue Léon Cogniet 13, veuve de M. le Baron Jean Baptiste Alfred) en bl. pour ceder et transférer créance',
 'Levyy par M^me Ronne Bloch épouseauguste Salonon p^t a Paris rue Beri 24 a son mari pour bendtre terrain a Paris avenue de Suffren et rue',
 'Fribe (a Mme Jeanne Estelle Séraphine Borget) époux deM. Joseph Paul) dessinateur a Paris 19 rue Montchanin 19 par le Credit foncier de500000^ pour 75 ans, 

## Reconnaissance d'entités nommées

On passe chaque index de notre liste dans la chaîne de traitement TAL.

In [6]:
doc = [nlp(entry) for entry in entries_list_without_linebreakes]

## Création du fichier texte

On crée un fichier texte contenant nos annotations au format BIOES.

In [7]:
with open("out/FRAN_0025_5038_L-0_htr_cer_13_bioes.txt", "w+", encoding="UTF-8") as fh:
    for entry in doc:
        for sent in entry.sentences:
            for token in sent.tokens:
                BIO_line = token.text + ' ' + token.ner
                fh.write(BIO_line)
                fh.write('\n')